<a href="https://colab.research.google.com/github/armansheikhibiglari/PRACTICE-2/blob/main/NLPCLASSIFICATION_exam_assinment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia
!pip install requests

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=090bfdd2f27d8dcae7726154489c2826f50aba09d543d34fffb27053c25ee699
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [2]:
import wikipedia
import nltk
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer as stemmer
from nltk import FreqDist
from nltk.classify import apply_features
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import re

In [3]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import nltk
nltk.download('punkt_tab')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import nltk
print(nltk.data.path)


['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [7]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Initialize the stemmer
stemmer = PorterStemmer()

def stem_tokenize_sp_case(text):
    # Tokenize the text
    tokens_word = word_tokenize(text)

    # Apply stemming to each token and join them back into a string
    stem_text = ' '.join([stemmer.stem(word) for word in tokens_word])

    return stem_text


In [8]:
import re
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK resources
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

# Preprocessing function
def preprocess_text(text):
    # Remove special characters and HTML tags
    text = re.sub(r"<.*?>", " ", text)  # Remove HTML tags
    text = re.sub(r"[^a-zA-Z ]", " ", text)  # Remove non-alphabetic characters

    # Lowercase
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Stopword removal
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens)

# Function to fetch data from Wikipedia API
def fetch_wikipedia_data(categories, label, max_articles=100):
    articles = []
    base_url = "https://en.wikipedia.org/w/api.php"

    for category in categories:
        params = {
            "action": "query",
            "list": "categorymembers",
            "cmtitle": f"Category:{category}",
            "cmlimit": max_articles,
            "format": "json",
        }

        response = requests.get(base_url, params=params)
        data = response.json()

        for page in data.get("query", {}).get("categorymembers", []):
            if "title" in page:
                articles.append((page["title"], label))

    return articles

# Fetching geographic and non-geographic articles
geographic_categories = ["Geography", "Maps"]
non_geographic_categories = ["History", "Engineering", "Art"]

data = fetch_wikipedia_data(geographic_categories, label="geographic")
data += fetch_wikipedia_data(non_geographic_categories, label="non-geographic")

# Create a DataFrame
df = pd.DataFrame(data, columns=["text", "label"])

# Preprocess the text data
df["cleaned_text"] = df["text"].apply(preprocess_text)

# Feature extraction
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["cleaned_text"]).toarray()
y = df["label"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Naive Bayes classifier
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)

print("Naive Bayes Classification Report:")
print(classification_report(y_test, nb_predictions))

# Logistic Regression classifier
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)

print("Logistic Regression Classification Report:")
print(classification_report(y_test, lr_predictions))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Naive Bayes Classification Report:
                precision    recall  f1-score   support

    geographic       0.83      0.95      0.88        40
non-geographic       0.94      0.79      0.86        38

      accuracy                           0.87        78
     macro avg       0.88      0.87      0.87        78
  weighted avg       0.88      0.87      0.87        78

Logistic Regression Classification Report:
                precision    recall  f1-score   support

    geographic       0.81      0.97      0.89        40
non-geographic       0.97      0.76      0.85        38

      accuracy                           0.87        78
     macro avg       0.89      0.87      0.87        78
  weighted avg       0.89      0.87      0.87        78

